In [104]:
import requests
import urllib3
import secrets
import pandas as pd
import polyline
import folium
from ast import literal_eval

In [105]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [106]:
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

print('Requesting Strava token... \n')
res = requests.post(auth_url, data=secrets.strava_payload, verify=False)
strava_access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + strava_access_token}

strava_requests_page_num = 1
all_activities = []

while True:
    strava_param = {'per_page' : 15, 'page' : strava_requests_page_num}
    strava_dataset = requests.get(activities_url, headers=header, params=strava_param).json()

    if len(strava_dataset) == 0:
        print('breaking out of Strava while loop because the response is zero, indicating no more activities.')
        break

    if all_activities:
        print('all activities is populated')
        all_activities.extend(strava_dataset)

    else:
        print('all activities is NOT populated')
        all_activities = strava_dataset

    strava_requests_page_num += 1

print('Total Activities: ', len(all_activities))

Requesting Strava token... 

all activities is NOT populated
all activities is populated
all activities is populated
all activities is populated
breaking out of Strava while loop because the response is zero, indicating no more activities.
Total Activities:  53


In [107]:
run_data = pd.DataFrame(data=all_activities)

In [108]:
columns = run_data.columns.to_list()
columns

['resource_state',
 'athlete',
 'name',
 'distance',
 'moving_time',
 'elapsed_time',
 'total_elevation_gain',
 'type',
 'sport_type',
 'workout_type',
 'id',
 'start_date',
 'start_date_local',
 'timezone',
 'utc_offset',
 'location_city',
 'location_state',
 'location_country',
 'achievement_count',
 'kudos_count',
 'comment_count',
 'athlete_count',
 'photo_count',
 'map',
 'trainer',
 'commute',
 'manual',
 'private',
 'visibility',
 'flagged',
 'gear_id',
 'start_latlng',
 'end_latlng',
 'average_speed',
 'max_speed',
 'has_heartrate',
 'heartrate_opt_out',
 'display_hide_heartrate_option',
 'elev_high',
 'elev_low',
 'upload_id',
 'upload_id_str',
 'external_id',
 'from_accepted_tag',
 'pr_count',
 'total_photo_count',
 'has_kudoed',
 'suffer_score',
 'average_watts',
 'kilojoules',
 'device_watts']

In [109]:
run_data.head()

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_watts,kilojoules,device_watts
0,2,"{'id': 8586088, 'resource_state': 1}",RRG : Rush Trail to Gray’s Arch Loop,6312.2,2374,2448,173.8,Run,Run,0.0,...,10150965476,3A315073-851C-4478-8D23-F4D440BAA704-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
1,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Lode Loop and Drive By Loop,5532.4,1884,1967,124.0,Run,Run,0.0,...,10143932040,7D43355C-FECB-473B-8263-E7C4BDB870DE-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
2,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Lode Loop and Sore Heel loop,7940.5,2861,2886,153.7,Run,Run,0.0,...,10131336663,4256B6D8-14F3-4F18-8DF3-440342D551CA-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
3,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Sore Heel Loop,7090.5,2341,2365,210.0,Run,Run,0.0,...,10124522361,0B1C68DC-4E71-4EBB-8217-F688A9582DCD-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
4,2,"{'id': 8586088, 'resource_state': 1}",Ten Sleep : French Cattle Ranch Approach,1594.6,1483,1483,186.4,Hike,Hike,NaN,...,10023599286,70EA4DF3-F218-4DB7-B818-E242D7EC3F45-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN


In [110]:
columns_to_drop = ['athlete',
                   'resource_state', 
                   'sport_type', 
                   'workout_type',
                   'location_city',
                   'location_state',
                   'location_country', 
                   'kudos_count', 
                   'comment_count', 
                   'athlete_count', 
                   'photo_count', 
                   'trainer', 
                   'commute', 
                   'manual', 
                   'private',
                   'visibility', 
                   'flagged', 
                   'gear_id', 
                   'has_heartrate', 
                   'heartrate_opt_out', 
                   'display_hide_heartrate_option', 
                   'from_accepted_tag', 
                   'total_photo_count', 
                   'has_kudoed', 
                   'average_watts', 
                   'kilojoules',
                   'achievement_count',
                   'device_watts',
                   'upload_id_str',
                   'upload_id',
                   'external_id', 
                   'suffer_score']

run_data.drop(columns=columns_to_drop, inplace=True)
columns = run_data.columns.to_list()

In [111]:
run_data = run_data.loc[run_data['type'] == 'Run']

In [112]:
run_data['distance_miles'] = round(run_data['distance'] * 0.00062137119, 2)
run_data['moving_time_minutes'] = round(run_data['moving_time'] / 60, 2)
run_data['moving_time_hours'] = round(run_data['moving_time'] / 3600, 2)

columns_to_drop = ['moving_time',
                   'elapsed_time']
run_data.drop(columns=columns_to_drop, inplace=True)

average_distance_miles = round(run_data['distance_miles'].mean(), 2)
print("Average Distance (miles):", average_distance_miles)

average_time_minutes = round(run_data['moving_time_minutes'].mean(), 2)
print("Average Time Ran (minutes):", average_time_minutes)

max_distance_ran = round(run_data['distance_miles'].max(), 2)
print("Longest Run:", max_distance_ran, "miles")

max_duration_mintues = round(run_data['moving_time_minutes'].max(), 2)
max_duration_hours = round(run_data['moving_time_hours'].max(), 2)
print("Longest Duration:", max_duration_mintues,"minutes. Converted to hours:", max_duration_hours)

total_distance_miles = round(run_data['distance'].sum() * 0.00062137119, 2)
print("Total Distance Covered to the date (miles):", total_distance_miles)

Average Distance (miles): 3.98
Average Time Ran (minutes): 40.56
Longest Run: 9.62 miles
Longest Duration: 117.57 minutes. Converted to hours: 1.96
Total Distance Covered to the date (miles): 147.18


In [113]:
columns = run_data.columns.to_list()

In [114]:
pmrp_run_data = run_data[run_data['name'].str.contains('PMRP', case=False, na=False)]
rrg_run_data = run_data[run_data['name'].str.contains('RRG', case=False, na=False)]
sore_heel_data = run_data[run_data['name'].str.contains('Sore Heel', case=False, na=False)]
lode_loop_data = run_data[run_data['name'].str.contains('Lode Loop', case=False, na=False)]
drive_by_loop_data = run_data[run_data['name'].str.contains('Drive By Loop', case=False,na=False)]

In [115]:
run_data['id']

0     9464901645
1     9458318004
2     9446383475
3     9439898989
6     9294421659
7     9294421645
8     9176575201
9     9159069458
10    9059541871
11    9017048707
13    8973585450
15    8885448119
16    8880411991
17    8874318870
20    8833532781
21    8804960417
23    8776433393
24    8697978353
25    8660873889
26    8654073455
27    8649852094
28    8632830532
29    8627995854
30    8616251867
31    8611260707
32    8601141195
34    8489474084
35    8441176813
36    8441005412
39    8212187320
41    8062853814
42    8023377719
43    7957898757
44    7952654943
48    6683286710
49    6576275626
50    6528238964
Name: id, dtype: int64

Why does the information go from 50 to 36?

In [116]:
run_data['map']

0     {'id': 'a9464901645', 'summary_polyline': 'qjw...
1     {'id': 'a9458318004', 'summary_polyline': 'aow...
2     {'id': 'a9446383475', 'summary_polyline': 'onw...
3     {'id': 'a9439898989', 'summary_polyline': 'inw...
6     {'id': 'a9294421659', 'summary_polyline': 'sn`...
7     {'id': 'a9294421645', 'summary_polyline': '_w_...
8     {'id': 'a9176575201', 'summary_polyline': 'ejw...
9     {'id': 'a9159069458', 'summary_polyline': '{mw...
10    {'id': 'a9059541871', 'summary_polyline': 'unw...
11    {'id': 'a9017048707', 'summary_polyline': 'kow...
13    {'id': 'a8973585450', 'summary_polyline': 'anw...
15    {'id': 'a8885448119', 'summary_polyline': 'oqw...
16    {'id': 'a8880411991', 'summary_polyline': 'yvy...
17    {'id': 'a8874318870', 'summary_polyline': 'wdl...
20    {'id': 'a8833532781', 'summary_polyline': 'k{y...
21    {'id': 'a8804960417', 'summary_polyline': 'wqw...
23    {'id': 'a8776433393', 'summary_polyline': 'qnw...
24    {'id': 'a8697978353', 'summary_polyline': 

In [117]:
all_map_data = pd.DataFrame(run_data['map'].to_list())
all_map_data
# all_map_data['id'] = all_map_data['id'].str.slice(start=1)
# all_map_data

,id,summary_polyline,resource_state
0,a9464901645,qjweFflr}NEAOOI_@Xi@VUJg@@_@zAmDQWOG?IKMG]MQIY...,2
1,a9458318004,aowdFvk}}N@KAEB?L_@Gg@CEOGEDI@ABE?MPAB@FCGFG@M...,2
2,a9446383475,onwdFzl}}NI{@Dw@GSm@FMTBW\YH]Cm@PY?ULY?OLK@_@K...,2
3,a9439898989,inwdFfm}}NBFONWHKEAEQ?QHWb@UBMd@qA`BQZK\GZKPER...,2
4,a9294421659,sn`lG`~~mSBBIUG_@K[A?OYE?Is@k@qB_@{@Om@Ys@MQOo...,2
5,a9294421645,_w_lGlyanSCMIu@S{@EGB[Ik@AYIIIU@MEEIc@CE?GOQGe...,2
6,a9176575201,ejweFrlr}NIQWOQAAEIAIKE?@FFHOOe@BGEKHSGQ?k@SOO...,2
7,a9159069458,{mwdFhl}}NEXUXo@?[^YPKCQt@eBvBeA|CY^Y|@GCQfAMD...,2
8,a9059541871,unwdFhj}}NACDG?GOa@M??HE@@EC?EHKHEAAFQHO?AACDW...,2
9,a9017048707,kowdFvm}}Nu@VuClDm@bBWTMv@MA[p@QhAOPD\m@r@[tAK...,2
